In [22]:
import pandas as pd
import numpy as np

In [23]:
data_path_2019 = "../00-raw/2019_top250.csv"
data_path_2020 = "../00-raw/2020_top250.csv"
data_path_2021 = "../00-raw/2021_top250.csv"
data_path_2022 = "../00-raw/2022_top250.csv"
data_path_2023 = "../00-raw/2023_top250.csv"

df_steam250_2019 = pd.read_csv(data_path_2019)
df_steam250_2020 = pd.read_csv(data_path_2020)
df_steam250_2021 = pd.read_csv(data_path_2021)
df_steam250_2022 = pd.read_csv(data_path_2022)
df_steam250_2023 = pd.read_csv(data_path_2023)

In [24]:
df_steam250_2019

,Rank,Game,Company,Score,Rating
0,1,People Playground 2019 Sandbox $9.99,NaN,6.04,98% 305 K
1,2,Slay the Spire 2019 Roguelike Deckbuilder $24.99,NaN,6.00,98% 205 K
2,3,Sekiro™: Shadows Die Twice - GOTY Edition 2019...,NaN,5.98,95% 343 K
3,4,Resident Evil 2 2019 Zombies $9.99 -75% $39.99,Demo,5.98,97% 189 K
4,5,Red Dead Redemption 2 2019 Open World $59.99,NaN,5.96,92% 844 K
...,...,...,...,...,...
245,246,Tailor Tales 2019 Otome Free,NaN,5.58,95% 999
246,247,Vanishing Realms™ 2019 Swordplay $19.99,VR,5.58,90% 2.57 K
247,248,Neon Boost 2019 Parkour Free,NaN,5.58,90% 2.47 K
248,249,The World is Your Weapon 2019 Roguelike $9.99,NaN,5.58,97% 667


In [25]:
years = range(2019, 2024)
source_dfs = {
    2019: df_steam250_2019,
    2020: df_steam250_2020,
    2021: df_steam250_2021,
    2022: df_steam250_2022,
    2023: df_steam250_2023,
}

price_pattern = r'(Free|\$\d+(?:\.\d{2})?(?:\s*-\d+%\s*\$\d+(?:\.\d{2})?)?)\s*$'
clean_by_year = {}

for year in years:
    df = source_dfs[year].copy()
    df['price_text'] = df['Game'].str.extract(price_pattern, expand=False)
    df['game_title'] = df['Game'].str.extract(fr'^(.*?)\s+{year}\b', expand=False)
    df['price_usd'] = (
        df['price_text']
        .str.extract(r'\$(\d+(?:\.\d{2})?)', expand=False)
        .astype(float)
        .fillna(0.0)
        .where(df['price_text'].ne('Free'), 0.0)
    )
    df['year'] = year
    clean_by_year[year] = df

clean_all = pd.concat(clean_by_year.values(), ignore_index=True)


In [26]:
for year in years:
    clean_by_year[year].drop(columns=['Game', 'Company', 'price_text'])
    new_order = ['Rank', 'year', 'game_title', 'Score', 'Rating', 'price_usd']
    clean_by_year[year] = clean_by_year[year][new_order]
    clean_by_year[year].columns = ['rank', 'year', 'game_title', 'score', 'rating', 'price_usd']

In [30]:
clean_by_year[2023].head(5)

,rank,year,game_title,score,rating,price_usd
0,1,2023,Baldur's Gate 3,6.07,97% 824 K,59.99
1,2,2023,Lethal Company,6.04,97% 502 K,9.99
2,3,2023,Resident Evil 4,5.99,97% 218 K,15.99
3,4,2023,DAVE THE DIVER,5.96,97% 146 K,10.99
4,5,2023,Pizza Tower,5.94,98% 70.1 K,19.99


In [31]:
clean_by_year[2023].dtypes

rank            int64
year            int64
game_title     object
score         float64
rating         object
price_usd     float64
dtype: object